In [4]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split

class NaiveBayesClassifier():
    def __init__(self):
        self.prior={}
        self.conditional={}
        
    def fit(self,X,y):
        self.classes=np.unique(y)
        for c in self.classes:
            self.prior[c]=np.mean(y == c)
            
        for feature in X.columns:
            self.conditional[feature]={}
            for c in self.classes:
                feature_value=X[feature][y == c]
                self.conditional[feature][c]={'mean':np.mean(feature_value),'std':np.std(feature_value)}
                
    def predict(self,X):
        y_pred=[]
        for _,sample in X.iterrows():
            probabilities={}
            for c in self.classes:
                probabilities[c]=self.prior[c]
                for feature in X.columns:
                    mean=self.conditional[feature][c]['mean']
                    std=self.conditional[feature][c]['std']
                    x=sample[feature]
                    probabilities[c]*=self.gaussian_pdf(x,mean,std)
                
            y_pred.append(max(probabilities,key=probabilities.get))
        return y_pred
    
    
    def gaussian_pdf(self,x,mean,std):
        exponent=np.exp(-((x-mean) ** 2) / (2 * std**2))
        return (1/(np.sqrt(2*np.pi) * std)) * exponent
    
    
    
df=pd.read_csv('Titanic-Dataset.csv')
df=df[['Survived','Pclass','Age','SibSp','Parch','Fare','Embarked']]
df['Age'].fillna(df['Age'].median(),inplace=True)
df['Fare'].fillna(df['Fare'].median(),inplace=True)
df['Embarked'].fillna(df['Embarked'].mode()[0],inplace=True)
df['Embarked']=df['Embarked'].map({'C':0,'Q':1,'S':2})

X=df.drop('Survived',axis=1)
y=df['Survived']

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

#train,test=train_test_split(df,test_size=0.2)

#X_train=train.drop('Survived',axis=1)
#y_train=train['Survived']
#X_test=test.drop('Survived',axis=1)
#y_test=test['Survived']


classifier=NaiveBayesClassifier()
classifier.fit(X_train,y_train)
y_pred=classifier.predict(X_test)

cm=confusion_matrix(y_test,y_pred)
print("Confusion Matrix:\n", cm)
accuracy1=np.mean(y_pred == y_test)
accuracy = np.sum(y_pred == y_test) / len(y_test)
print("Accuracy:",accuracy1)
print("Accuracy:",accuracy)
print("Cuurcay : ",accuracy_score(y_pred,y_test))
                
                
    
        
            

Confusion Matrix:
 [[88 17]
 [36 38]]
Accuracy: 0.7039106145251397
Accuracy: 0.7039106145251397
Cuurcay :  0.7039106145251397
